In [ ]:
import re

import nltk
import pandas as pd

nltk.download('punkt')

In [ ]:
dataset1 = pd.read_csv("datasets/paper_title_abstract.tsv", sep="\t", header=0)
dataset2 = pd.read_csv("datasets/task1_trainset.csv")
dataset3 = pd.read_csv("datasets/task1_public_testset.csv")

dataset1 = list(dataset1["Abstract"])
dataset2 = [ abstract.replace("$$$", "") for abstract in dataset2["Abstract"] ]
dataset3 = [ abstract.replace("$$$", "") for abstract in dataset3["Abstract"] ]

In [ ]:
total_dataset = dataset1 + dataset2 + dataset3
len(total_dataset)

In [ ]:
pattern = r"[(https?:\/\/)|(www\.)]+[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"

In [ ]:
from multiprocessing import Pool
from tqdm import *

def tokeinze(text):
    clean_url = re.sub(pattern, "", text)
    token = nltk.word_tokenize(clean_url)
    return " ".join(token)


with Pool(processes=8) as p:
    token_dataset = []
    for i, result in tqdm(enumerate(p.imap_unordered(tokeinze, total_dataset)), total=len(total_dataset)):
        token_dataset.append(result)

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

trainset, validset = train_test_split(token_dataset, test_size=0.05, random_state=42)
print(len(trainset), len(validset))

In [ ]:
train_text = "\n\n".join(trainset)
with open("datas/train.txt", "w") as writer:
    writer.write(train_text)
    
validset = "\n\n".join(validset)
with open("datas/valid.txt", "w") as writer:
    writer.write(validset)